# Emission Model SWDS

This little project is supposed to help you evaluate potential methane emissions from Solid Waste Disposal Sites according to the model used in the [CDM UNFCCC](https://cdm.unfccc.int/methodologies/PAmethodologies/tools/am-tool-04-v6.0.1.pdf/history_view). Complete the tasks a) and b) given in the notebook to obtain the ready to use tool. 

There are some nice illustrations in here, like heatmaps and stackplots, so I hope you have fun! I also put some comments in the code and a short explanation above each cell, so that the code can be better understood and modified. You can download the jupyter notebook or use the mybinder environment without having to install all the Python stuff.

Thanks to Allen Downey, whose book "[Modeling and Simulation in Python](https://greenteapress.com/wp/modsimpy/)" helped me a lot in the building process. I also used his [ModSim library](https://pypi.org/project/modsimpy/) in the project. Thanks also to Jan.

## 0. Importing modules
This cell makes sure that all packages used in the code are available.

In [ ]:
from modsim import *
import numpy as np
from matplotlib.pyplot import figure, plot, axis, grid, imshow, show, legend, gca
import pandas as pd
import seaborn as sns
%matplotlib inline

## 1. Defining model system

### 1.1 Waste systems
The composition of the waste and its characteristics are defined.

#### Task a) 
Find out the DOCj and the kj for the different waste types in the [CDM UNFCCC](https://cdm.unfccc.int/methodologies/PAmethodologies/tools/am-tool-04-v6.0.1.pdf/history_view) description. The climate in the city is tropical (MAT>20°C, wet MAP>1000mm).

In [ ]:
#Food waste
waste_charac_fw = System(
    label="Food waste",
    share=0.6,
    DOCj="put value here (p.v.h.)",
    kj="p.v.h.")


#Garden and park waste
waste_charac_gpw = System(
    label="G&P waste",
    share=0.1,
    DOCj="p.v.h.",
    kj="p.v.h.")


#Wood waste
waste_charac_ww = System(
    label="Wood waste",
    share=0.03,
    DOCj="p.v.h.",
    kj="p.v.h.")


#Paper
waste_charac_p = System(
    label="Paper",
    share=0.1,
    DOCj="p.v.h.",
    kj="p.v.h.")


#Textiles
waste_charac_t = System(
    label="Textiles",
    share=0.03,
    DOCj="p.v.h.",
    kj="p.v.h.")


#Other
waste_charac_o = System(
    label="Other",
    share=0.14,
    DOCj="p.v.h.",
    kj="p.v.h.")



Plotting waste composition

In [ ]:
y = np.array([waste_charac_fw.share, waste_charac_gpw.share, waste_charac_ww.share, waste_charac_p.share,
             waste_charac_t.share, waste_charac_o.share])
mylabels = ["Food waste", "Garden and Park waste", "Wood waste", "Paper", "Textiles", "Others"]
plt.figure(figsize=(10,6), dpi=80)
plt.title('Waste composition')
plt.pie(y, labels = mylabels)
plt.show() 


### 1.2 Model parameters
The specific model parameters are defined in here. Then, they are summarized as "Const_multi".

#### Task b) 
Find out the model parameters for the use case in the [CDM UNFCCC](https://cdm.unfccc.int/methodologies/PAmethodologies/tools/am-tool-04-v6.0.1.pdf/history_view) description.

Assumptions: 
- No gas capture exists
- Anaerobic managed landfill
- Not covered with soil or compost

In [ ]:
#Defining model

model = System(
    f = "p.v.h.", #fraction of methane captured and flared/combusted/used
    phi = "p.v.h.", # model correction factor
    F = "p.v.h.", #  Fraction of CH4 in LFG 
    DOCf = "p.v.h.",  #fraction of DOC dissimilated 
    OX = "p.v.h.", # Oxidation factor 
    MCF = "p.v.h.", # methane correction factor 
    )


#Combining constant parameters
Const_multi = model.phi*(1-model.f)*(1-model.OX)*(16/12)*model.F*model.DOCf*model.MCF

### 1.3 Time frame
The next thing to do is to define how long the disposal site will be used and the time period of emission prognosis.

In [ ]:
#t is period of waste disposal
t_0= 2021
t_end = 2040 
year_array = range(t_0,t_end)

#y is period of prognosis
y_0 = 2021
y_end = 2070
year_of_prognosis_array = range(y_0,y_end)



### 1.4 City data
This cell defines parameters like population and its growth, waste production per person, service coverage, ... . Then, the total amount of waste that is produced in each year is calculated.

In [ ]:
#Waste per year

#given data
pop = 1000000 
pop_growth = 0.02
r = 0.7 #rate of waste collection after informal waste collection
service_coverage = 0.7 #kg

#Introducing a TimeSeries element with initial conditation, 
#because "t in year_array" always stops one step too early
W_0 = pop*r*service_coverage*365/1000
results_wg = TimeSeries()
results_wg[t_0] = W_0


#loop to fill TimeSeries waste generation
for t in year_array:
    results_wg[t+1] = round(results_wg[t] + results_wg[t] * pop_growth)
    
#print(results_wg)
d = {'Total waste landfilled [t]': results_wg}
df = pd.DataFrame(data=d)


#waste generation bar plot
ax = df.plot(legend=False, kind="bar")
ax.set_xlabel("Year [a]")
ax.set_ylabel("Total waste landfilled [t]")
show(ax)

## 2. Defining prognosis formulas
In this cell, the two formulas for the actual prognosis are defined. Later, those will be used in a simple  way. The first formula calculates the total methane emissions in each year of prognosis for the different waste types and displays them in a table. This data will be used for the stack plot later on. The second formula generates heatmaps, that display how much and how long waste that has been landfilled in a certain year emits methane.

In [ ]:
#Defining formula for the generation of emission matrix for different waste types and heatmaps.

def generate_emission_matrix(waste_charac, Const_multi):

    #Introducing TimesSeries() element for the sum of the emissions of the different waste types
    sum_results_E = TimeSeries()
    
    # looping through prognosis years to get emissions in a specific year of prognosis for the amount of wastes
    for y in year_of_prognosis_array:
        #Introducing TimesSeries() element for the emissions per year of prognosis
        #for specific years of the different waste types
        results_E = TimeSeries()
        E_0 = Const_multi * results_wg[t_0] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y-t_0))*(1-exp(-waste_charac.kj))
        results_E[t_0] = E_0
        # looping through years 
        for t in year_array:
            if y-(t+1) >= 0:
                results_E[t+1] = Const_multi * results_wg[t+1] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y-(t+1)))*(1-exp(-waste_charac.kj))
            else:
                results_E[t+1] = "N.A."
        
        
        
        #Putting all results in one big array
        if y == year_of_prognosis_array[0]:
            matrix = np.array([results_E])
        else:
            matrix = np.concatenate((matrix, [results_E]))
        
        #Generate the sum of emissions for each prognosis year
        a = 0
        for i in results_E:
            if i != "N.A.":
                a+=i
        sum_results_E[y] = a
        
    # Calculating the emissions for the last year of prognosis, the loop above does not calculate that 
    results_E = TimeSeries()
    E_end = Const_multi * results_wg[t_0] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y_end-t_0))*(1-exp(-waste_charac.kj))
    results_E[t_0] = E_end
    for t in year_array:
            if y_end-(t+1) >= 0:
                results_E[t+1] = Const_multi * results_wg[t+1] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y_end-(t+1)))*(1-exp(-waste_charac.kj))
            else:
                results_E[t+1] = "N.A."
    a = 0
    for i in results_E:
        if i != "N.A.":
            a+=i
    sum_results_E[y_end] = a
    matrix = np.concatenate((matrix, [results_E]))
    
    return sum_results_E



def generate_emission_heatmaps_matrix(waste_charac, Const_multi):
    heatmap_data = dict()
    #Introducing TimesSeries() element for the sum of the emissions of the different waste types
    sum_results_E = TimeSeries()
    
    # looping through prognosis years to get emissions in a specific year of prognosis for the amount of wastes
    for y in year_of_prognosis_array:
        #Introducing TimesSeries() element for the emissions per year of prognosis
        #for specific years of the different waste types
        results_E = TimeSeries()
        E_0 = Const_multi * results_wg[t_0] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y-t_0))*(1-exp(-waste_charac.kj))
        results_E[t_0] = E_0
        # looping through years 
        for t in year_array:
            if y-(t+1) >= 0:
                results_E[t+1] = Const_multi * results_wg[t+1] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y-(t+1)))*(1-exp(-waste_charac.kj))
            else:
                results_E[t+1] = "N.A."
        
        
        #Putting all results in one big array
        if y == year_of_prognosis_array[0]:
            matrix = np.array([results_E])
        else:
            matrix = np.concatenate((matrix, [results_E]))
        heatmap_data[y] = results_E
        
        
    # Calculating the emissions for the last year of prognosis, the loop above does not calculate that 
    results_E = TimeSeries()
    E_end = Const_multi * results_wg[t_0] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y_end-t_0))*(1-exp(-waste_charac.kj))
    results_E[t_0] = E_end
    for t in year_array:
            if y_end-(t+1) >= 0:
                results_E[t+1] = Const_multi * results_wg[t+1] * waste_charac.share * waste_charac.DOCj * exp(-waste_charac.kj*(y_end-(t+1)))*(1-exp(-waste_charac.kj))
            else:
                results_E[t+1] = "N.A."
    heatmap_data[y_end] = results_E
    
    
    #plotting heatmap
    df = pd.DataFrame(data = heatmap_data)
    df = df.transpose()
    df = df.replace("N.A.", 0)  
    plt.subplots(figsize=(10,5))
    sns.heatmap(df, cbar_kws={'label': 'Emissions [t CH4]'}) #for same color range add , vmin=0, vmax=1800 in heatmap(...)
    ax.set_title(f"Emissions from {waste_charac.label} [t CH4]",fontsize =20)
    plt.xlabel("Year of landfilling [a]") 
    plt.ylabel("Year of prognosis [a]")
    plt.title(f"Heatmap {waste_charac.label}")
    plt.show()

## 3. Modeling
Now we can generate our results via plots and tables. It takes a while if you run the cells, be patient.. :) 

### 3.1 Heatmaps

In [ ]:
generate_emission_heatmaps_matrix(waste_charac_fw, Const_multi)
generate_emission_heatmaps_matrix(waste_charac_gpw, Const_multi)
generate_emission_heatmaps_matrix(waste_charac_ww, Const_multi)
generate_emission_heatmaps_matrix(waste_charac_p, Const_multi)
generate_emission_heatmaps_matrix(waste_charac_t, Const_multi)
generate_emission_heatmaps_matrix(waste_charac_o, Const_multi)

### 3.2 Emission matrix

In [ ]:
y_fw = generate_emission_matrix(waste_charac_fw, Const_multi)
y_gpw = generate_emission_matrix(waste_charac_gpw, Const_multi)
y_ww = generate_emission_matrix(waste_charac_ww, Const_multi)
y_p = generate_emission_matrix(waste_charac_p, Const_multi)
y_t = generate_emission_matrix(waste_charac_t, Const_multi)
y_o = generate_emission_matrix(waste_charac_o, Const_multi)

In [ ]:
d = {'E Food waste [t CH4]': y_fw, 
     'E G&P waste [t CH4]': y_gpw,
     'E Wood waste [t CH4]': y_ww,
     'E Paper [t CH4]': y_p,
     'E Textiles [t CH4]': y_t,
     'E Others [t CH4]': y_o}

df = pd.DataFrame(data=d)


df = df.replace(0.000000, "N.A.")
df= df.style.highlight_max(color = 'lightgreen')
df

### 3.3 Emission stackplot

In [ ]:
x = range(y_0,y_end+1)
plt.figure(figsize=(10,6), dpi=80)
plt.stackplot(x, y_fw, y_gpw, y_ww,y_p,y_t)
plt.legend(["Food waste", "Garden and Park waste","Wood waste", "Paper", "Textiles"])
plt.title('CH4 Emissions from waste types')
plt.grid('on')
plt.xlabel('Years [a]')
plt.ylabel('CH4 Emissions [t CH4]')
plt.show()

## 4. Export to Excel

By running the cell below, you can export the summary matrix to an Excel file on your computer. If you are using mybinder, click on the folder button in the upper left corner below the jupyter logo, there you can find the file for you to download. You can copy the values and use them in your Landfill excel sheet.

In [ ]:
df.to_excel("emission_matrix.xlsx")